## Implementação de Naive Bayes

O algoritmo de Naive Bayes é baseado em probabilidade condicional. Este é um dos mais simples algoritmos de classificação. Inicialmente, deve-se separar todas as classes, e então para cada _feature_, é necessário encontrar a probabilidade para seus valores.
### Definir as configurações

In [110]:
import numpy as np
import pandas as pd
import math


#df = pd.read_csv("./diabetes.csv")
df = pd.read_csv("./wine.csv", index_col = False)

#target = 'Outcome'
target = 'Alcohol'

features = df.columns[df.columns != target]
classes = df[target].unique()
data_length = len(df.index)

k = 5
num_examples = math.floor(data_length/k)    

### Construir os Dataset

In [111]:
train = []
test = []

for i in range(0,k):
    test.append(df[i*num_examples : (i+1)*num_examples])
    train.append(df.drop(test[i].index))
    
test[1]

,Alcohol,Malic,Ash,Alcalinity,Magnesium,phenols,Flavanoids,Nonflavanoid,Proanthocyanins,Color,Hue,OD280,Proline
35,1,13.48,1.81,2.41,20.5,100,2.70,2.98,0.26,1.86,5.10,1.04,3.47
36,1,13.28,1.64,2.84,15.5,110,2.60,2.68,0.34,1.36,4.60,1.09,2.78
37,1,13.05,1.65,2.55,18.0,98,2.45,2.43,0.29,1.44,4.25,1.12,2.51
38,1,13.07,1.50,2.10,15.5,98,2.40,2.64,0.28,1.37,3.70,1.18,2.69
39,1,14.22,3.99,2.51,13.2,128,3.00,3.04,0.20,2.08,5.10,0.89,3.53
40,1,13.56,1.71,2.31,16.2,117,3.15,3.29,0.34,2.34,6.13,0.95,3.38
41,1,13.41,3.84,2.12,18.8,90,2.45,2.68,0.27,1.48,4.28,0.91,3.00
42,1,13.88,1.89,2.59,15.0,101,3.25,3.56,0.17,1.70,5.43,0.88,3.56
43,1,13.24,3.98,2.29,17.5,103,2.64,2.63,0.32,1.66,4.36,0.82,3.00
44,1,13.05,1.77,2.10,17.0,107,3.00,3.00,0.28,2.03,5.04,0.88,3.35


### Cálculo das Probabilidades

Aqui será caluclada as probabilidades de cada classe, e serão armazenadas em uma estrutura de dicionário 

```
dict: 
  keys: classe
  values: dict: 
        keys: atributo
        values: dict:
              keys: valor
              values: probabilidade do valor
```

Deste modo, a probabilidade de cada classe poderá ser facilmente acessada.

In [112]:
probs = []
probcl = []

for i in range(0,k):
    probs.append({})
    probcl.append({})
    
    for x in classes:
        dfcl = df[df[target]==x][features]
        clsp = {}
        tot = len(dfcl)
        for col in dfcl.columns:
            colp = {}
            for val,cnt in dfcl[col].value_counts().iteritems():
                pr = cnt/tot
                colp[val] = pr
            clsp[col] = colp
        probs[i][x] = clsp
        probcl[i][x] = len(dfcl)/len(df)
        
def probabs(i, x):
    # i - Iteração K-Fold
    # x - DataFrame Pandas com índices
    
    if not isinstance(x,pd.Series):
        raise IOError("Arg must of type Series")
        
    probab = {}
    for cl in classes:
        pr = probcl[i][cl]
        for col,val in x.iteritems():
            try:
                pr *= probs[i][cl][col][val]
            except KeyError:
                pr = 0
        probab[cl] = pr
    return probab

def classify(i, x):
    # i - Iteração K-Fold
    # x - DataFrame Pandas com índices
    
    probab = probabs(i, x)
    mx = 0
    mxcl = ''
    for cl,pr in probab.items():
        if pr > mx:
            mx = pr
            mxcl = cl
            
    return mxcl

### Treinamento

In [118]:
b = []

for l in range(0,k):
    for j in train[l].index:
        # print(classify(l, train[l].loc[j,features]), train[l].loc[j])
        # print('')
        b.append([])
        b[l].append(classify(l, train[l].loc[j,features]) == train[l].loc[j,target])
    
    print(sum(b[l]),"corretos de ",len(train[l]))
    print("Precisão:", sum(b[l])/len(train[l]))
    print('')

143 corretos de  143
Precisão: 1.0

143 corretos de  143
Precisão: 1.0

143 corretos de  143
Precisão: 1.0

143 corretos de  143
Precisão: 1.0

143 corretos de  143
Precisão: 1.0



### Teste

In [119]:
a = []

for l in range(0,k):
    for j in test[l].index:
        #print(classify(l, test[l].loc[j,features]), test[l].loc[j])
        #print('')
        a.append([])
        a[l].append(classify(l, test[l].loc[j,features]) == test[l].loc[j,target])
    
    print(sum(a[l]),"corretos de",len(test[l]))
    print("Precisão:", sum(a[l])/len(test[l]))
    print('')

35 corretos de 35
Precisão: 1.0

35 corretos de 35
Precisão: 1.0

35 corretos de 35
Precisão: 1.0

35 corretos de 35
Precisão: 1.0

35 corretos de 35
Precisão: 1.0

